# Time Series Analysis

Time series analysis is the statistical analysis of data that is collected at regular intervals over time. It is used to understand the patterns in the data, forecast future values, and identify outliers.

### Benefits of time series analysis
- It can help to identify patterns and trends in data that would not be visible otherwise.
- It can be used to forecast future values of the data, which can be used to make better decisions.
- It can help to identify outliers and anomalies, which can be investigated further.


### Where is time series analysis used?
- In `Finance`, to forecast stock prices, currency exchange rates, and other financial market data.
- In `Retail`, to forecast sales, demand, and inventory levels.

# There are several categories in the Superstore sales data, we start from time series analysis and forecasting for furniture sales.

### Import all necessary Libraries

In [25]:
# Ignore warning related to pandas_profiling
import warnings

# advanced tools for creating and working with iterators
import itertools

# Fundamental package for linear algebra and multidimensional arrays
import numpy as np

# Python package for data manipulation and analysis
import pandas as pd

# Library for Visualization
import matplotlib

# Plotting library with Matplotlib for Python
import matplotlib.pyplot as plt

# Python module that provides classes and functions for the estimation of many different statistical models
import statsmodels.api as sm

# Use FiveThirtyEight style plots
plt.style.use("fivethirtyeight")
# Suppress all warnings
warnings.filterwarnings("ignore")

# Format the size of x and y axis tick labels
matplotlib.rcParams["axes.labelsize"] = 14
# Format the size of the x axis tick labels
matplotlib.rcParams["xtick.labelsize"] = 12
# Format the size of the y axis tick labels
matplotlib.rcParams["ytick.labelsize"] = 12
# Change the color of the text to black
matplotlib.rcParams["text.color"] = "k"

#### Let's read our dataset

In [26]:
# Let's read our data and store it in a dataframe called df
df = pd.read_excel("../Time_Series_Analysis/dataset/Sample - Superstore.xls")

In [27]:
# Let's see what's inside our data
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [28]:
# Let's see hom many entries we have for each category
df["Category"].value_counts()

Category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64

As we can see, category `Office Supplies` has maximum sales. So, let's use this category for our further analysis. We'll create a separate dataset for `Office Supplies` and analyze it.

In [29]:
# Create a DataFrame called office_supply containing the "Office Supplies" category
office_supply = df.loc[df["Category"] == "Office Supplies"]

# Print the first 5 rows of office_supply
office_supply.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620,2,0.0,6.8714
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164
6,7,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.280,4,0.0,1.9656
8,9,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.504,3,0.2,5.7825
9,10,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.900,5,0.0,34.4700


In [30]:
# Let's check starting and ending date of office supply orders in our data
office_supply["Order Date"].min(), office_supply["Order Date"].max()

(Timestamp('2014-01-03 00:00:00'), Timestamp('2017-12-30 00:00:00'))

So we have our `Office supply` orders from 2014 to 2017. Let's dive into it and analyze it.

#### Cleaning and Preparing Data

In [31]:
# Let's see if we have any missing values in our office supply data
office_supply.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

So we don't have any null values in our dataset. Let's check for duplicate values.

In [32]:
# Check for duplicate data
office_supply.duplicated().sum()

0

So we don't have duplicate values as well. 

As we only want `Order Date` and `Sales` for our analysis. Let's drop other columns. 

In [33]:
# Drop unnecessary columns
columns_to_remove = [
    "Row ID",
    "Order ID",
    "Ship Date",
    "Ship Mode",
    "Customer ID",
    "Customer Name",
    "Segment",
    "Country",
    "City",
    "State",
    "Postal Code",
    "Region",
    "Product ID",
    "Category",
    "Sub-Category",
    "Product Name",
    "Quantity",
    "Discount",
    "Profit",
]

office_supply = office_supply.drop(columns_to_remove, axis=1)

# Sort the DataFrame by 'Order Date'
office_supply = office_supply.sort_values("Order Date")

# Display the resulting DataFrame
office_supply

,Order Date,Sales
7980,2014-01-03,16.448
740,2014-01-04,272.736
739,2014-01-04,11.784
741,2014-01-04,3.540
1759,2014-01-05,19.536
...,...,...
5091,2017-12-30,3.024
1297,2017-12-30,20.720
908,2017-12-30,52.776
1296,2017-12-30,13.904


In [34]:
# Aggregate sales data by date and reset index to create a time series of total sales per day

office_supply = office_supply.groupby(
    "Order Date")["Sales"].sum().reset_index()
office_supply

,Order Date,Sales
0,2014-01-03,16.448
1,2014-01-04,288.060
2,2014-01-05,19.536
3,2014-01-06,685.340
4,2014-01-07,10.430
...,...,...
1143,2017-12-26,814.594
1144,2017-12-27,13.248
1145,2017-12-28,1091.244
1146,2017-12-29,282.440


In [35]:
# Let's make 'Order Date' as the index for time-based analysis

office_supply = office_supply.set_index("Order Date")
office_supply.index

DatetimeIndex(['2014-01-03', '2014-01-04', '2014-01-05', '2014-01-06',
               '2014-01-07', '2014-01-09', '2014-01-10', '2014-01-13',
               '2014-01-16', '2014-01-18',
               ...
               '2017-12-21', '2017-12-22', '2017-12-23', '2017-12-24',
               '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28',
               '2017-12-29', '2017-12-30'],
              dtype='datetime64[ns]', name='Order Date', length=1148, freq=None)

In [40]:
office_supply


,Sales
Order Date,
2014-01-03,16.448
2014-01-04,288.060
2014-01-05,19.536
2014-01-06,685.340
2014-01-07,10.430
...,...
2017-12-26,814.594
2017-12-27,13.248
2017-12-28,1091.244


Our current datetime data can be tricky to work with, therefore, we will use the averages daily sales value for that month instead, and we are using the start of each month as the timestamp.

In [42]:
# Resampling and calculating monthly mean sales starting from 2014.

monthly_mean_sales = office_supply["Sales"].resample("MS").mean()
monthly_mean_sales["2014":]

Order Date
2014-01-01     285.357647
2014-02-01      63.042588
2014-03-01     391.176318
2014-04-01     464.794750
2014-05-01     324.346545
2014-06-01     588.774409
2014-07-01     756.060400
2014-08-01     541.879143
2014-09-01    1015.677704
2014-10-01     267.078815
2014-11-01     959.372714
2014-12-01     692.556231
2015-01-01     129.198571
2015-02-01     335.504188
2015-03-01     690.545522
2015-04-01     502.342320
2015-05-01     364.549440
2015-06-01     560.407737
2015-07-01     205.214739
2015-08-01     558.814667
2015-09-01     772.230680
2015-10-01     361.392083
2015-11-01     757.790357
2015-12-01     540.055800
2016-01-01     331.230125
2016-02-01     357.597368
2016-03-01     693.877240
2016-04-01     462.932478
2016-05-01     449.489724
2016-06-01     436.072400
2016-07-01     587.474727
2016-08-01     344.605385
2016-09-01     830.847786
2016-10-01     678.408083
2016-11-01     787.972231
2016-12-01    1357.055929
2017-01-01     967.013136
2017-02-01     389.882737
2